<a href="https://colab.research.google.com/github/jyotidabass/Transfer-Learning-Convolutions-and-Object-Localisation-in-Keras/blob/main/Transfer_Learning%2C_Convolutions%2C_and_Object_Localisation_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#importing the required libraries
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pathlib
import tensorflow as tf
import PIL
import PIL.Image
import matplotlib.image as img
from tensorflow.keras.preprocessing import image
import matplotlib.cm as cm
from IPython.display import Image, display  
import pandas as pd
import os
import cv2

In [23]:
data_dir = pathlib.Path('/content/drive/MyDrive/Two class/Train')

In [24]:
#reshaping the image size according to requirement of input to the implemented model
IMAGE_SIZE = (224,224)

In [25]:
# creating the training dataset
train_dataset = image_dataset_from_directory('/content/drive/MyDrive/Two class/Train',image_size=IMAGE_SIZE)

Found 426 files belonging to 2 classes.


In [26]:
# creating the validation dataset
validation_dataset = image_dataset_from_directory('/content/drive/MyDrive/Two class/Train',image_size=IMAGE_SIZE)

Found 426 files belonging to 2 classes.


In [27]:
# creating the test dataset
test_dataset = image_dataset_from_directory('/content/drive/MyDrive/Two class/Train',image_size=IMAGE_SIZE)

Found 426 files belonging to 2 classes.


In [28]:
#importing the required libraries
import os
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten,Dropout,GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

In [29]:
#performing data augmentation , since the dataset isn't huge and it also helps reduce overfitting
#Reference : https://www.tensorflow.org/tutorials/images/data_augmentation
data_augmentation = tf.keras.Sequential([
  RandomFlip('horizontal'),
  RandomRotation(0.25),
])

In [31]:
# Creating the base model from the pre-trained ResNetv2 model
# by defining the include_top=False argument we are excluding the dense classification head(top) of the model
base_model = tf.keras.applications.ResNet50V2(include_top=False,weights='imagenet',input_shape=(224,224,3))

In [32]:
# freezing the base convolutional model, so that these weights aren't updated during training
for layer in base_model.layers:
	layer.trainable = False

In [33]:
# A summary of all the layers that are present in the base model Resnet50
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

In [34]:
inputs = tf.keras.Input(shape=(224,224, 3)) #reshaping the inputs to (224,224,3) to input to the model
# Performing data augmentation of the inputs
x = data_augmentation(inputs) 
# Since the pixel values are in range [0-255], pre-processing them according to the requirment of the model
x = tf.keras.applications.resnet_v2.preprocess_input(x)
# adding the base model to our model
x = base_model(x, training=False)
#Applying the GlobalAveragePooling function here
x = GlobalAveragePooling2D()(x)
# adding a dense layer with 1024 neurons and relu activation to the model.
x = Dense(1024, activation='relu')(x) 
# adding dropout layer to avoid overfitting
x = Dropout(0.3)(x)
# adding a dense layer with 512 neurons and relu activation to the model.
x = Dense(512, activation='relu')(x)
# adding dropout layer similarly to avoid overfitting
x = Dropout(0.3)(x)
# finally applying a dense layer to convert the model output to a single prediction per image
# Since this prediction will be treated as a logit, we do not need an activation function here
outputs = Dense(1)(x)
#defining the model
model = tf.keras.Model(inputs, outputs)

In [35]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
#compiling the model using the standard adam optimizer and binarycrossentropy loss,since our problem is a binary classification problem
model.compile(optimizer=Adam(lr=0.005),
              loss=BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [36]:
## A summary of all the layers that are present in the our model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0   

In [37]:
#training our model and saving it to a variable 'history'
history = model.fit(train_dataset,epochs=10,validation_data=validation_dataset)

Epoch 1/10
14/14 [==============================] - 270s 13s/step - loss: 3.3798 - accuracy: 0.5376 - val_loss: 0.8971 - val_accuracy: 0.5657
Epoch 2/10
14/14 [==============================] - 157s 11s/step - loss: 0.7602 - accuracy: 0.5563 - val_loss: 0.6605 - val_accuracy: 0.5775
Epoch 3/10
14/14 [==============================] - 161s 11s/step - loss: 0.6956 - accuracy: 0.5986 - val_loss: 0.6492 - val_accuracy: 0.5915
Epoch 4/10
14/14 [==============================] - 161s 11s/step - loss: 0.6417 - accuracy: 0.6174 - val_loss: 0.6176 - val_accuracy: 0.6221
Epoch 5/10
14/14 [==============================] - 165s 11s/step - loss: 0.6395 - accuracy: 0.6432 - val_loss: 0.6000 - val_accuracy: 0.6362
Epoch 6/10
14/14 [==============================] - 161s 11s/step - loss: 0.6143 - accuracy: 0.6362 - val_loss: 0.5681 - val_accuracy: 0.6808
Epoch 7/10
14/14 [==============================] - 168s 12s/step - loss: 0.6252 - accuracy: 0.6432 - val_loss: 0.5602 - val_accuracy: 0.6573
Epoch 

In [39]:
import itertools
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

all_labels=[]  #list to hold all true labels of the images
all_predictions=[] #list to hold all the predicted labels of the images
for image_batch, labels_batch in test_dataset:
  #appending the true labels of the batch to all_labels list
  all_labels.append(labels_batch.numpy())
  #predicting the labels for each test_dataset batch and flattening the tensor
  preds = model.predict_on_batch(image_batch).flatten()
  # Applying a sigmoid since our model returns logits
  preds = tf.nn.sigmoid(preds)
  preds = tf.where(preds < 0.5, 0, 1)
  #appending the predicted labels of the batch to all_predictioons list
  all_predictions.append(preds.numpy())

#Converting list of lists to a single list of all_labels
all_labels = [list(itertools.chain(item)) for item in all_labels]
all_labels = list(itertools.chain.from_iterable(all_labels))

#Converting list of lists to a single list of all_predictions
all_predictions = [list(itertools.chain(item)) for item in all_predictions]
all_predictions = list(itertools.chain.from_iterable(all_predictions))

loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

#Confusion matrix using sklearn's confusion matrix for the test dataset
print('Confusion Matrix')
print(confusion_matrix(all_labels,all_predictions))

#Classification report on the test dataset
print('Classification Report')
target_names = ['Kapha', 'Vata']
print(classification_report(all_labels,all_predictions, target_names=target_names))

14/14 [==============================] - 79s 5s/step - loss: 0.4779 - accuracy: 0.7488
Test accuracy : 0.748826265335083
Confusion Matrix
[[216  25]
 [ 66 119]]
Classification Report
              precision    recall  f1-score   support

       Kapha       0.77      0.90      0.83       241
        Vata       0.83      0.64      0.72       185

    accuracy                           0.79       426
   macro avg       0.80      0.77      0.77       426
weighted avg       0.79      0.79      0.78       426



In [53]:
inputs = tf.keras.Input(shape=(224,224, 3)) #reshaping the inputs to (224,224,3) to input to the model
# Since the pixel values are in range [0-255], pre-processing them according to the requirment of the model
x = tf.keras.applications.resnet_v2.preprocess_input(inputs)
# base model
x = base_model(x, training=False)
# applying GlobalMaxPooling2D method to extract a scalar output from each neuron in the final layer of the base model
x = GlobalMaxPooling2D()(x)
# adding the softmax layer
outputs = Dense(2,activation="softmax")(x)
#defining the model
model1 = tf.keras.Model(inputs, outputs)

In [54]:
from keras import models
import keras
# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model1.layers])
# get name of global max pooling layer
layer_name = list(layer_dict.keys())[-2]
 
#getting global max pooling layers output
layer_output = layer_dict[layer_name].output
# creating a model with model1s input and global max pooling layers output
activation_model =keras.Model(inputs=model1.inputs, outputs=layer_output)
activation_model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_3 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_3 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_max_pooling2d_1 (Glo  (None, 2048)             0         
 balMaxPooling2D)                                                
                                                           

In [55]:
# get the scalar output from global max pooling layer
activations = activation_model.predict(validation_dataset)
# get softmax output from dense layer which contains two neurons as this is a binary classification task 
# 1st neuron output is assigned to activation2 variable 
activations2 = model1.predict(validation_dataset)[:,0]
# 2nd neurons output is assigned to activation3 variable 
activations3 = model1.predict(validation_dataset)[:,1]

# correlation for each neuron from globalMaxPooling layer is calculated with output neurons in dense layer
#corr0 holds correlations of class label 0
corr0=[]
# loop is iterated over all the neurons from globalmaxpooling layer
for i in range(np.shape(activations)[1]):
  #correlation is calculated using corrcoef from numpy library
  corr0.append(np.corrcoef(activations[:,i],activations2)[0,1])

#corr1 holds correlations of class label 1
corr1=[]
# loop is iterated over all the neurons from globalmaxpooling layer
for i in range(np.shape(activations)[1]):
  #correlation is calculated using corrcoef from numpy library
  corr1.append(np.corrcoef(activations[:,i],activations3)[0,1])

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [43]:
# all the neurons that has correlation above the threshold are identified and stored in neurons1 list
neurons1=[]
#loop is run over all the correlations 
for i in range(0,len(corr1)-1):
  #neurons are filtered with a threshold
  if(corr1[i]>0.33):
    neurons1.append(i)
    # printing neurons that are correlated with class label 1
    print("neuron : "+ str(i) +" is strongly correlated to input image label 1 with correlation : "+str(corr1[i]))
neurons0=[]
#loop is run over all the correlations 
for i in range(0,len(corr0)-1):
  #neurons are filtered with a threshold
  if(corr0[i]>0.15):
    neurons0.append(i)
    # printing neurons that are correlated with class label 1
    print("neuron : "+ str(i) +" is strongly correlated to input image 0 label with correlation : "+str(corr0[i]))

In [44]:
#model3 is created from base model input and output base model output
model3=keras.Model(inputs=base_model.inputs,outputs=base_model.outputs)
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

#model3 is compiled to commit the changes made
model3.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [45]:
# superimposing base models generated heatmap output with actual image
def superimposed_heat_map(image_path,neuron,cam_path="cam.jpg"):
  img = keras.preprocessing.image.load_img(image_path, target_size=(224,224))
  x = keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.resnet_v2.preprocess_input(x)
  # considering only the output produced by specific neuron to generate heat map
  heatmap = model3.predict(x)[0,:,:,neuron] 
  #plotting the heatmap
  plt.matshow(heatmap, cmap='viridis')
  plt.show()
  
  #loading the original image
  img = keras.preprocessing.image.load_img(image_path)
  img = keras.preprocessing.image.img_to_array(img)

  # rescaling the heatmap to a range 0-255
  heatmap = np.uint8(255 * heatmap)   
  # Use jet colormap to colorize heatmap
  jet = cm.get_cmap("jet")                              

  # Use RGB values of the colormap
  jet_colors = jet(np.arange(256))[:, :3]
  jet_heatmap = jet_colors[heatmap]

  # Create an image with RGB colorized heatmap
  jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
  jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
  jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

  # Superimpose the heatmap on original image
  superimposed_image = jet_heatmap * 0.7 + img            
  superimposed_image = keras.preprocessing.image.array_to_img(superimposed_image)
  superimposed_image = superimposed_image.resize((300,300)) 
  # Save the superimposed image
  superimposed_image.save(cam_path)    
  # display gradcam
  display(Image(cam_path))

In [50]:
#image is downloaded from given url
img_path='/content/drive/MyDrive/Two class/Train/Tongue Image - Kapha/16305812861725004033431300973848 - kishan kumar.jpg'
for neuron in neurons0:
  #2d output for each strongly correlated neurons from base layer are produced and that 2d output is used to superimpose the image
  superimposed_heat_map(img_path,neuron=neuron ,cam_path="16305812861725004033431300973848 - kishan kumar.jpg")

In [52]:
#image is downloaded from given url
img_path='/content/drive/MyDrive/Two class/Train/Tongue Image - Vata/16401893939488735646405764724660 - pinakin Desai.jpg'
for neuron in neurons0:
  #2d output for each strongly correlated neurons from base layer are produced and that 2d output is used to superimpose the image
  superimposed_heat_map(img_path,neuron=neuron ,cam_path="16401893939488735646405764724660 - pinakin Desai.jpg")